In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura ainda não divulgou uma nova edição da Imersão IA com Google Gemini. A última edição ocorreu em fevereiro de 2024.

Para ficar por dentro de futuras edições, recomendo:

*   **Acompanhar as redes sociais da Alura:** Eles costumam anunciar novos eventos e cursos por lá.
*   **Verificar a página de eventos da Alura:** Procure na página oficial da Alura por uma seção de eventos ou imersões.
*   **Assinar a newsletter da Alura:** Assim você recebe as novidades diretamente no seu e-mail.

Assim que tiverem uma data confirmada, certamente divulgarão por esses canais.

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Existe alguma Imersão IA com Google Gemini da Alura acontecendo no momento?',
    config = {"tools": [{"google_search":{}}]}
    # Inserir a tool de busca do Google ###############################################
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 Sim, de acordo com os resultados da pesquisa, a Alura e o Google se uniram para oferecer uma Imersão IA com o Google Gemini. A Imersão IA é um curso online e gratuito, sem pré-requisitos, que visa ensinar como usar a IA do Google Gemini para criar projetos práticos e automatizar tarefas.

Com base nas informações mais recentes, a Imersão IA mais recente ocorreu entre os dias 12 e 16 de maio de 2025. Além disso, em 13 de maio de 2025, houve uma Masterclass sobre os primeiros passos com Python para usar IA no dia a dia, como parte da Imersão IA da Alura e Google.


In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA Google Gemini']
Páginas utilizadas na resposta: alura.com.br, tecmundo.com.br, youtube.com



In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
%pip -q install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca informações no Google",
        tools=[google_search],
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do Google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele  não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscados, você deve:
        usar a ferramenta de busca do google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais informações
        sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos a serem
        abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Esqueceu de digitar o tópico? 🤔")
else:
  print(f"Maravilha! Vamos criar o post sobre {topico}")
  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("\n--- Resultado do Agente 1 (Buscador) ---\n")
  display(to_markdown(lancamentos_buscados))
  print("---------------------------------------------------")

  plano_de_post = agente_planejador(topico, lancamentos_buscados)
  print("\n--- Resultado do Agente 2 (Planejador) ---\n")
  display(to_markdown(plano_de_post))
  print("---------------------------------------------------")

  rascunho_de_post = agente_redator(topico, plano_de_post)
  print("\n--- Resultado do Agente 3 (Redator) ---\n")
  display(to_markdown(rascunho_de_post))
  print("---------------------------------------------------")

  post_final = agente_revisor(topico, rascunho_de_post)
  print("\n--- Resultado do Agente 4 (Revisor) ---\n")
  display(to_markdown(post_final))
  print("---------------------------------------------------")


🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: cpi das bets
Maravilha! Vamos criar o post sobre cpi das bets

--- Resultado do Agente 1 (Buscador) ---



> Para encontrar os lançamentos mais relevantes relacionados à CPI das Bets no último mês, farei algumas pesquisas no Google.
> 
> 
> Com base nas notícias mais recentes sobre a CPI das Bets, os lançamentos mais relevantes incluem:
> 
> 1.  **Depoimentos de Influenciadores:** A CPI tem focado em ouvir influenciadores digitais e celebridades que promovem plataformas de apostas online. Recentemente, Virginia Fonseca e Rico Melquiades prestaram depoimentos (13 e 14 de maio, respectivamente). Outros nomes como Gusttavo Lima, Wesley Safadão, Gkay, e Jojo Todynho também estão na mira da CPI, seja como convocados ou convidados.
> 2.  **Investigação sobre Vínculos Financeiros:** A CPI investiga se os influenciadores lucram com as perdas dos apostadores e se há irregularidades nos recebimentos ou vínculos com empresas barradas pelo governo. Há também foco na relação entre plataformas de apostas e possíveis crimes financeiros, como lavagem de dinheiro.
> 3.  **Operação Game Over:** O influenciador Rico Melquiades foi questionado sobre seu envolvimento na Operação Game Over, que investiga a promoção irregular de jogos de azar online.
> 4.  **Foco na Saúde Mental:** A CPI também está debatendo os efeitos do vício em apostas online e a importância de procurar ajuda especializada, com alertas sobre os efeitos psicológicos da prática compulsiva.
> 5.  **Prorrogação da CPI:** A CPI das Bets tem previsão de encerramento em 14 de junho, com remotas chances de prorrogação.
> 
> 


---------------------------------------------------

--- Resultado do Agente 2 (Planejador) ---



> Okay, com base nos lançamentos mais recentes sobre a CPI das Bets, vou criar um plano de conteúdo com os pontos mais relevantes para abordar em posts nas redes sociais.
> 
> 
> Com base nas pesquisas, o tema mais relevante para abordar em um post sobre a CPI das Bets é:
> 
> **O Impacto da CPI das Bets na Saúde Mental dos Apostadores e a Responsabilidade dos Influenciadores.**
> 
> **Pontos Relevantes:**
> 
> *   **Crescente preocupação com a saúde mental:** A CPI está debatendo os efeitos do vício em apostas online e a importância de procurar ajuda especializada, com alertas sobre os efeitos psicológicos da prática compulsiva (ludopatia).
> *   **Influenciadores sob investigação:** Figuras públicas que promovem plataformas de apostas estão sendo investigadas para determinar se lucram com as perdas dos apostadores e se há irregularidades nos recebimentos ou vínculos com empresas barradas pelo governo.
> *   **Necessidade de regulamentação:** A CPI busca criar regras para limitar as apostas no país e prevenir o vício em jogos.
> *   **Apoio e tratamento:** Destacar a importância de informar, escutar e não julgar os viciados em jogos, mencionando que a ludopatia é uma doença tratável com apoio e terapia.
> 
> **Plano de Conteúdo para o Post:**
> 
> 1.  **Introdução:**
>     *   Contextualizar a CPI das Bets e seu objetivo de investigar os impactos das apostas online nas famílias brasileiras.
>     *   Destacar a crescente preocupação com a saúde mental dos apostadores e o papel dos influenciadores nesse cenário.
> 2.  **Desenvolvimento:**
>     *   **O impacto na saúde mental:**
>         *   Explicar o que é ludopatia (vício em jogos) e seus sintomas (ansiedade, depressão, culpa, impulsividade, insônia, etc.).
>         *   Mencionar que a ludopatia é uma doença reconhecida pela medicina e que muitos só procuram ajuda após perder dinheiro, relacionamentos e autoestima.
>     *   **A responsabilidade dos influenciadores:**
>         *   Discutir como a publicidade feita por influenciadores pode normalizar ou encobrir práticas questionáveis nas apostas online.
>         *   Citar os nomes de influenciadores que estão sendo investigados pela CPI (Virginia Fonseca, Rico Melquiades, Gusttavo Lima, Wesley Safadão, Gkay, Jojo Todynho, etc.) e os motivos de suas convocações.
>         *   Explicar a diferença entre investigados e testemunhas na CPI.
>     *   **A busca por regulamentação:**
>         *   Informar que a CPI busca criar regras para limitar as apostas no país e prevenir o vício em jogos.
>         *   Mencionar a prorrogação da CPI e a possibilidade de novas oitivas.
> 3.  **Conclusão:**
>     *   Reforçar a importância de cuidar da saúde mental e buscar ajuda caso sofra com o vício em jogos.
>     *   Oferecer informações sobre onde encontrar apoio e tratamento (grupos de apoio, terapia individual, etc.).
>     *   Incentivar a discussão sobre a regulamentação das apostas online e a responsabilidade dos influenciadores.
> 4.  **Chamada para ação (CTA):**
>     *   Perguntar aos seguidores se eles conhecem alguém que sofre com o vício em jogos e incentivá-los a compartilhar informações sobre o tema.
>     *   Convidar os seguidores a expressarem suas opiniões sobre a regulamentação das apostas online e a responsabilidade dos influenciadores.
> 
> **Hashtags:** \#CPIdasBets \#ApostasOnline \#SaudeMental \#Ludopatia \#Influenciadores \#Regulamentacao \#VicioEmJogos
> 


---------------------------------------------------

--- Resultado do Agente 3 (Redator) ---



> Okay, Alura! Prontos para mergulhar em um tema quente que tá dando o que falar? 🔥
> 
> ---
> 
> **RASCUNHO DE POST PARA INSTAGRAM**
> 
> **(Imagem/Carrossel Sugerido: Uma arte gráfica com o título "CPI das Bets: O Jogo da Saúde Mental e a Responsabilidade dos Influencers" ou imagens que remetam a atenção, debate, e cuidado com a saúde mental, evitando imagens diretas de apostas ou cassinos.)**
> 
> ---
> 
> **Legenda:**
> 
> Você tem acompanhado a CPI das Bets? 🧐 Mais do que investigar fraudes, ela está acendendo um alerta importantíssimo sobre a **saúde mental** dos apostadores e o papel dos **influenciadores digitais** nesse universo.
> 
> A preocupação é crescente: o vício em apostas online, conhecido como **ludopatia**, é uma doença séria. 🧠 Ela pode desencadear ansiedade, depressão, culpa e levar a perdas financeiras e de relacionamentos. Muitos só buscam ajuda quando o estrago já é grande.
> 
> E os influenciadores? 🤔 Figuras públicas como Virginia Fonseca, Gusttavo Lima, Jojo Todynho e outros estão sendo convocados pela CPI. A investigação busca entender se há lucro sobre as perdas dos seguidores e se as promoções de plataformas de apostas são transparentes e éticas. É crucial diferenciar quem é chamado como testemunha e quem é investigado.
> 
> Por isso, a CPI também quer criar **regras mais claras** para as apostas no Brasil, buscando proteger os cidadãos e prevenir o vício. 📜
> 
> **O mais importante:**
> Se você ou alguém que conhece está enfrentando dificuldades com apostas:
> 1.  **INFORME-SE:** Entenda os riscos e os sinais do vício.
> 2.  **ESCUTE SEM JULGAR:** Acolhimento é fundamental.
> 3.  **BUSQUE AJUDA:** Ludopatia tem tratamento! Terapia e grupos de apoio podem fazer toda a diferença.
> 
> Este é um debate necessário sobre responsabilidade, informação e, acima de tudo, cuidado com as pessoas. ❤️
> 
> 💬 E aí, qual a sua opinião sobre a regulamentação das bets e a responsabilidade dos influenciadores? Você conhece alguém que precisa ler sobre isso? Compartilhe e vamos conversar! 👇
> 
> \#CPIdasBets \#SaudeMental \#ApostasOnline \#Regulamentacao


---------------------------------------------------

--- Resultado do Agente 4 (Revisor) ---



> Opa! O rascunho tem uma base muito boa, abordando pontos cruciais da CPI das Bets e conectando com a saúde mental e a responsabilidade dos influenciadores. A estrutura está lógica e a preocupação com a imagem também é pertinente.
> 
> No entanto, tenho algumas sugestões para deixar o post ainda mais alinhado com o público jovem (18-30 anos) e otimizado para o Instagram:
> 
> **Problemas e Sugestões de Melhoria:**
> 
> 1.  **"Okay, Alura! Prontos para mergulhar em um tema quente que tá dando o que falar? 🔥"**
>     *   **Problema:** Essa frase parece uma instrução para você (Alura) ou um início de conversa informal que não se encaixa como abertura de um post público. O público não vai entender quem é "Alura" nesse contexto.
>     *   **Sugestão:** Remover essa frase completamente e começar o post com um gancho mais direto e que fale com o público.
> 
> 2.  **Tom e Linguagem:**
>     *   **Problema:** Embora o tom geral seja bom, algumas frases podem ser um pouco formais ou densas para um post de Instagram que visa engajamento rápido.
>     *   **Sugestão:** Usar uma linguagem ainda mais próxima, com gírias leves (se apropriado para a marca) e frases mais curtas e diretas. Quebrar parágrafos maiores.
> 
> 3.  **Concisão e Clareza Inicial:**
>     *   **Problema:** O primeiro parágrafo da legenda é bom, mas podemos torná-lo mais direto ao ponto.
>     *   **Sugestão:** Veja a reescrita abaixo.
> 
> 4.  **Estrutura para Escaneabilidade:**
>     *   **Problema:** Textos longos no Instagram podem afastar o leitor.
>     *   **Sugestão:** Usar mais quebras de linha, bullet points (como você já fez na parte da ajuda, o que é ótimo!) ou emojis para separar ideias e facilitar a leitura dinâmica.
> 
> ---
> 
> **Sugestão de Rascunho Revisado:**
> 
> **(Imagem/Carrossel Sugerido: Manter a sugestão original – arte gráfica "CPI das Bets: O Jogo da Saúde Mental e a Responsabilidade dos Influencers" ou imagens que remetam a atenção, debate, e cuidado com a saúde mental, evitando imagens diretas de apostas ou cassinos.)**
> 
> ---
> 
> **Legenda Revisada:**
> 
> E aí, galera! 🎮 Já tão ligados na CPI das Bets? Esse rolê vai MUITO além de investigar fraudes e tá botando o dedo na ferida sobre um assunto mega importante: a **saúde mental** de quem aposta e qual o papel dos **influencers** nisso tudo. 🧐
> 
> Sério, o alerta tá ligado! 🚨 O vício em apostas online, a tal da **ludopatia**, é uma doença real e pode bagunçar a vida:
> *   Ansiedade e deprê nas alturas 😥
> *   Culpa constante 😔
> *   Prejuízos financeiros GIGANTES 💸
> *   Problemas nos relacionamentos 💔
> Muitos só se tocam do problema quando o estrago já tá feito.
> 
> E os influencers nessa história? 🤔 Nomes como Virginia Fonseca, Gusttavo Lima, Jojo Todynho e outros foram chamados pra dar um papo na CPI. A ideia é sacar se rola lucro em cima da perda dos seguidores e se a publi das bets é feita na transparência e na ética. É chave entender quem tá lá como testemunha e quem tá sendo investigado, viu?
> 
> Por isso, a CPI também quer botar ordem na casa e criar **regras mais claras** pras apostas no Brasil. Tudo pra proteger a galera e evitar que o vício se espalhe. 📜
> 
> **O MAIS IMPORTANTE:** 👇
> Se você ou alguém que você conhece tá passando perrengue com apostas:
> 1.  **SE INFORMA:** Entenda os riscos e os sinais do vício. Conhecimento é poder!
> 2.  **ESCUTA SEM JULGAR:** Apoio e acolhimento fazem toda a diferença.
> 3.  **BUSQUE AJUDA PROFISSIONAL:** Ludopatia tem tratamento! Terapia e grupos de apoio salvam.
> 
> Esse debate é super necessário e fala sobre responsa, informação e, principalmente, cuidado com as pessoas. ❤️
> 
> 💬 Manda a real: qual sua opinião sobre a regulamentação das bets e a responsa dos influencers? Conhece alguém que precisa ler este post? Compartilha aí e bora trocar uma ideia! 👇
> 
> \#CPIdasBets \#SaudeMental \#ApostasOnline \#Regulamentacao \#Influenciadores \#ResponsabilidadeDigital
> 
> ---
> 
> **Principais Mudanças e Justificativas:**
> 
> *   **Remoção da saudação "Okay, Alura!"**: Essencial para um post público.
> *   **Linguagem mais jovem e dinâmica**: Uso de "galera", "tão ligados", "rolê", "botando o dedo na ferida", "mega importante", "alerta tá ligado", "tal da ludopatia", "bagunçar a vida", "nas alturas", "prejuízos GIGANTES", "se tocam", "dar um papo", "sacar se rola", "publi", "na transparência", "chave", "botar ordem na casa", "proteger a galera", "passando perrengue", "conhecimento é poder", "salvam", "super necessário", "responsa", "manda a real", "trocar uma ideia".
> *   **Emojis mais integrados**: Para quebrar o texto e adicionar apelo visual.
> *   **Bullet points para os impactos da ludopatia**: Melhora a escaneabilidade.
> *   **Frases mais curtas e diretas**: Facilitam a leitura rápida.
> *   **Adição de hashtags relevantes**: Como #Influenciadores e #ResponsabilidadeDigital.
> 
> Com essas alterações, acredito que o post ficará ainda mais atrativo e eficaz para o público-alvo no Instagram!


---------------------------------------------------
